In [4]:
import numpy as np
import open3d as o3d
from skimage import measure

# --- Load & (optional) quick clean of the source mesh ---
voxels = np.load("data/sphere.npy")
verts, faces, normals, values = measure.marching_cubes(voxels, spacing=(1.0,1.0,1.0), level=0.5)
src = o3d.geometry.TriangleMesh()
src.vertices = o3d.utility.Vector3dVector(verts)
src.triangles = o3d.utility.Vector3iVector(faces)
src.remove_duplicated_vertices(); src.remove_degenerate_triangles()
src.remove_duplicated_triangles(); src.remove_unreferenced_vertices()
src.remove_non_manifold_edges()  # doesn't guarantee repair, but helps
src.compute_vertex_normals()

# o3d.visualization.draw_geometries([src])

# --- 1) Near-uniform resampling on the surface ---
# Choose a target edge length h (units = your model units)
h = 0.5  # example; tune
surf_area = src.get_surface_area()
# ~2 samples per triangle of edge ~h
n_pts = int(2 * surf_area / (np.sqrt(3)/4 * h*h))
pcd = src.sample_points_poisson_disk(n_pts, init_factor=5, pcl=None)

# --- 2) Normals: robust estimation + consistent orientation ---
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=2.5*h, max_nn=60)
)
# Make normals globally consistent so Poisson has a single "inside"
pcd.orient_normals_consistent_tangent_plane(k=50)

# --- 3) Poisson reconstruction (inherently watertight) ---
mesh_raw, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9, scale=1.1, linear_fit=False
)

# --- 4) Trim low-density floating sheets WITHOUT opening the surface ---
# Strategy A: keep the triangle cluster that contains the highest-density region
densities = np.asarray(densities)
tri_density = densities[np.asarray(mesh_raw.triangles).mean(axis=1).astype(int)]

# Remove very low density faces (outer sheets); use a conservative threshold
th = np.quantile(tri_density, 0.02)  # keep top 98% by density
tris_keep = np.where(tri_density >= th)[0]
mesh_trim = mesh_raw.select_by_index(tris_keep, retain_triangles=True)

# Keep only the largest connected triangle component (prevents holes)
cluster_ids, counts, _ = mesh_trim.cluster_connected_triangles()
cluster_ids = np.asarray(cluster_ids)
largest = int(np.argmax(counts))
mesh_trim = mesh_trim.select_by_index(
    np.where(cluster_ids == largest)[0], retain_triangles=True
)

# Optional: spatial crop to the original bounding box with a small margin
aabb = src.get_axis_aligned_bounding_box()
aabb = aabb.scale(1.05, aabb.get_center())  # 5% margin
mesh_trim = mesh_trim.crop(aabb)

# --- 5) Clean, smooth (non-shrinking), orient, verify ---
mesh_trim.remove_duplicated_vertices(); mesh_trim.remove_degenerate_triangles()
mesh_trim.remove_duplicated_triangles(); mesh_trim.remove_unreferenced_vertices()
mesh_trim.remove_non_manifold_edges()
mesh_trim = mesh_trim.filter_smooth_taubin(number_of_iterations=15)  # mild, non-shrinking
mesh_trim.orient_triangles()  # consistent winding
mesh_trim.compute_vertex_normals()

print("Watertight:", mesh_trim.is_watertight())
print("Edge-manifold (no boundary):", mesh_trim.is_edge_manifold(allow_boundary=False))
print("Vertex-manifold:", mesh_trim.is_vertex_manifold())
print("Self-intersections:", mesh_trim.is_self_intersecting())


o3d.visualization.draw_geometries([mesh_trim])

TypeError: select_by_index(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cuda.pybind.geometry.TriangleMesh, indices: list[int], cleanup: bool = True) -> open3d.cuda.pybind.geometry.TriangleMesh

Invoked with: TriangleMesh with 190674 points and 381344 triangles., array([     0,      1,      2, ..., 381341, 381342, 381343],
      shape=(373718,)); kwargs: retain_triangles=True